# Inspect container dwell times

In [ ]:
import os
import pathlib
import ipywidgets as widgets
import pandas as pd
from IPython.display import Markdown
import matplotlib.pyplot as plt
from matplotlib import gridspec

In [ ]:
folder_of_this_jupyter_notebook = pathlib.Path.cwd()
export_folder = os.path.join(
    folder_of_this_jupyter_notebook,
    os.pardir,
    "data",
    "exports"
)
folders = [
    folder
    for folder in os.listdir(export_folder)
    if os.path.isdir(
        os.path.join(
            export_folder,
            folder
        )
    )
]

dropdown_field = widgets.Dropdown(
    options=list(reversed(folders)),  # always show the newest first
    description='',
    layout={'width': 'max-content'}
)
dropdown_label = widgets.Label(value="Select the exported output: ")
display(widgets.HBox([dropdown_label, dropdown_field]))

In [ ]:
path_to_selected_exported_content = os.path.join(
    export_folder,
    dropdown_field.value
)

print("Working with directory " + path_to_selected_exported_content)

## Load containers

In [ ]:
path_to_containers = os.path.join(
    path_to_selected_exported_content,
    "containers.csv"
)
print(f"Opening {path_to_containers}")
df_containers = pd.read_csv(path_to_containers, index_col="id", dtype={
    "delivered_by_truck": "Int64",
    "picked_up_by_truck": "Int64",
    "delivered_by_large_scheduled_vehicle": "Int64",
    "picked_up_by_large_scheduled_vehicle": "Int64"
})

df_containers

In [ ]:
df_containers.groupby(by="delivered_by_large_scheduled_vehicle").count()

## Load scheduled vehicles

In [ ]:
path_to_deep_sea_vessels = os.path.join(
    path_to_selected_exported_content,
    "deep_sea_vessels.csv"
)

path_to_feeders = os.path.join(
    path_to_selected_exported_content,
    "feeders.csv"
)

path_to_barges = os.path.join(
    path_to_selected_exported_content,
    "barges.csv"
)

path_to_trains = os.path.join(
    path_to_selected_exported_content,
    "trains.csv"
)

scheduled_vehicle_file_paths = {
    "deep_sea_vessels": path_to_deep_sea_vessels,
    "feeders": path_to_feeders,
    "barges": path_to_barges,
    "trains": path_to_trains
}

for name, path in scheduled_vehicle_file_paths.items():
    print("Check file exists for vehicle " + name + ".")
    assert os.path.isfile(path)

print("All files exist.")

In [ ]:
for name, path in list(scheduled_vehicle_file_paths.items()):
    print("Check file size for vehicle " + name)
    size_in_bytes = os.path.getsize(path)
    if size_in_bytes <= 4:
        print("    This file is empty, ignoring it in the analysis from now on")
        del scheduled_vehicle_file_paths[name]

In [ ]:
scheduled_vehicle_dfs = {
    name: pd.read_csv(path, index_col=0, parse_dates=["scheduled_arrival"])
    for name, path in scheduled_vehicle_file_paths.items()
}

for name, df in scheduled_vehicle_dfs.items():
    display(Markdown("#### " + name))
    scheduled_vehicle_dfs[name]["vehicle_type"] = name
    display(scheduled_vehicle_dfs[name].sort_values(by="scheduled_arrival"))

In [ ]:
df_large_scheduled_vehicle = pd.concat(
    scheduled_vehicle_dfs.values()
)
df_large_scheduled_vehicle.sort_index(inplace=True)
df_large_scheduled_vehicle.info()
df_large_scheduled_vehicle

Plot arrival pattern.

In [ ]:
plt.figure(figsize=(15, 3))

x, y, z = [], [], []
y_axis = []

y_scaling_factor = 2

for i, (name, df) in enumerate(scheduled_vehicle_dfs.items()):
    y_axis.append((i/y_scaling_factor, name))
    if len(df) == 0:
        continue
    arrivals_and_capacity = df[["scheduled_arrival", "moved_capacity"]]
    for _, row in arrivals_and_capacity.iterrows():
        event = row["scheduled_arrival"]
        moved_capacity = row["moved_capacity"]
        x.append(event)
        y.append(i / y_scaling_factor)
        z.append(moved_capacity / 20)

plt.xticks(rotation=45)
plt.yticks(*list(zip(*y_axis)))
plt.scatter(x, y, s=z, color='gray')
plt.ylim([-0.5, 1.5])
plt.show()

In [ ]:
vehicle_to_teu_to_deliver = {}
vehicle_to_teu_to_pickup = {}

for i, container in df_containers.iterrows():
    teu = container["length"] / 20
    assert 1 <= teu <= 2.5

    if container["delivered_by"] != "truck":
        vehicle = container["delivered_by_large_scheduled_vehicle"]
        if vehicle not in vehicle_to_teu_to_deliver.keys():
            vehicle_to_teu_to_deliver[vehicle] = 0
        vehicle_to_teu_to_deliver[vehicle] += teu

    if container["picked_up_by"] != "truck":
        vehicle = container["picked_up_by_large_scheduled_vehicle"]
        if vehicle not in vehicle_to_teu_to_pickup.keys():
            vehicle_to_teu_to_pickup[vehicle] = 0
        vehicle_to_teu_to_pickup[vehicle] += teu

vehicle_to_teu_to_deliver, vehicle_to_teu_to_pickup

In [ ]:
s_delivery = pd.Series(vehicle_to_teu_to_deliver)
s_pickup = pd.Series(vehicle_to_teu_to_pickup)
df_large_scheduled_vehicle["capacity_delivery"] = s_delivery
df_large_scheduled_vehicle["capacity_pickup"] = s_pickup
df_large_scheduled_vehicle

Let's visualize in red if our transportation capacities were exceeded

In [ ]:
ax = df_large_scheduled_vehicle.plot.scatter(
    x="capacity_in_teu",
    y="capacity_delivery"
)

df_large_scheduled_vehicle.loc[
    df_large_scheduled_vehicle["capacity_in_teu"] < df_large_scheduled_vehicle["capacity_delivery"]
].plot.scatter(
    x="capacity_in_teu",
    y="capacity_delivery",
    ax=ax,
    color="r"
)
plt.show()

ax = df_large_scheduled_vehicle.plot.scatter(
    x="moved_capacity",
    y="capacity_delivery"
)
df_large_scheduled_vehicle.loc[
    df_large_scheduled_vehicle["moved_capacity"] < df_large_scheduled_vehicle["capacity_delivery"]
].plot.scatter(
    x="moved_capacity",
    y="capacity_delivery",
    color="r",
    ax=ax
)

plt.show()

free_delivery_capacity = df_large_scheduled_vehicle["moved_capacity"] - df_large_scheduled_vehicle["capacity_delivery"]
free_delivery_capacity.plot.hist()
plt.show()

In [ ]:
ax = df_large_scheduled_vehicle.plot.scatter(
    x="capacity_in_teu",
    y="capacity_pickup"
)
plt.show()

df_large_scheduled_vehicle.loc[
    df_large_scheduled_vehicle["capacity_in_teu"] < df_large_scheduled_vehicle["capacity_pickup"]
].plot.scatter(
    x="capacity_in_teu",
    y="capacity_pickup",
    ax=ax,
    color="r"
)
plt.show()

ax = df_large_scheduled_vehicle.plot.scatter(
    x="moved_capacity",
    y="capacity_pickup"
)
transportation_buffer = 1.2
df_large_scheduled_vehicle.loc[
    df_large_scheduled_vehicle["moved_capacity"] * transportation_buffer < df_large_scheduled_vehicle["capacity_pickup"]
].plot.scatter(
    x="moved_capacity",
    y="capacity_pickup",
    color="r",
    ax=ax
)

plt.show()

free_delivery_capacity = df_large_scheduled_vehicle["moved_capacity"] * 1.2 - df_large_scheduled_vehicle["capacity_pickup"]
plt.xlabel("Difference between moved capacity and the capacity of picked up containers")
free_delivery_capacity.plot.hist()
plt.show()

If there was no red dot in any of the graphs above, the following should work smoothly.

In [ ]:
for large_scheduled_vehicle_id in df_large_scheduled_vehicle.index:
    delivered_teu = vehicle_to_teu_to_deliver.get(large_scheduled_vehicle_id, 0)
    picked_up_teu = vehicle_to_teu_to_pickup.get(large_scheduled_vehicle_id, 0)
    capacity_in_teu = df_large_scheduled_vehicle.loc[large_scheduled_vehicle_id, "capacity_in_teu"]
    assert delivered_teu <= capacity_in_teu, f"{delivered_teu} is more than {capacity_in_teu} for vehicle "\
                                             f"with id {large_scheduled_vehicle_id}"
    assert picked_up_teu <= capacity_in_teu, f"{picked_up_teu} is more than {capacity_in_teu} for vehicle "\
                                             f"with id {large_scheduled_vehicle_id}"

## Load trucks

In [ ]:
path_to_trucks = os.path.join(
    path_to_selected_exported_content,
    "trucks.csv"
)
assert os.path.isfile(path_to_trucks)

In [ ]:
df_truck = pd.read_csv(
    path_to_trucks, index_col=0,
    parse_dates=[
        # Pickup
        "planned_container_pickup_time_prior_berthing",
        "realized_container_pickup_time",

        # Delivery
        "planned_container_delivery_time_at_window_start",
        "realized_container_delivery_time"
    ])
df_truck

In [ ]:
assert len(df_truck[df_truck["picks_up_container"] & pd.isna(df_truck["realized_container_pickup_time"])]) == 0, \
       "If a truck picks up a container, it should always have a realized container pickup time"

assert len(df_truck[df_truck["delivers_container"] & pd.isna(df_truck["realized_container_delivery_time"])]) == 0, \
       "If a truck deliver a container, it should always have a realized container delivery time"

assert len(df_truck[~(df_truck["delivers_container"] | df_truck["picks_up_container"])]) == 0, \
       "There is no truck that neither delivers or picks up a container"

In [ ]:
arrivals = pd.DataFrame({"x": x, "y": y})
arrivals.set_index("x").plot(marker=".", linestyle="None")
plt.show()

arrivals = arrivals.set_index("x")

In [ ]:
plt.figure(figsize=(15, 5))

container_deliveries_by_truck = df_truck.groupby(
    pd.Grouper(key='realized_container_delivery_time', freq='H')
).count().fillna(0)

ax = container_deliveries_by_truck["delivers_container"].plot()
ax.set_title("Number of trucks arriving in each hour that deliver a container")

ax2 = arrivals.plot(color='red', ax=ax, marker=".", linestyle="None", secondary_y=True)
ticks, labels = list(zip(*y_axis))

ax2.set_yticks(ticks)
ax2.set_yticklabels(labels)

plt.show()

In [ ]:
fig = plt.figure(figsize=(10, 5))
# set height ratios for subplots
gs = gridspec.GridSpec(2, 1, height_ratios=[2, 1]) 

# the upper subplot
ax1 = plt.subplot(gs[0])

plt.title("Relationship of vessels and truck arrivals")

ax1.set_ylabel("Number trucks per hour")
ax12 = container_deliveries_by_truck["delivers_container"].plot(ax=ax1, color="dimgray")
ax12.set_xlim([pd.Timestamp("2021-06-15"), pd.Timestamp(pd.Timestamp("2021-08-15"))])

# the lower subplot
ax2 = plt.subplot(gs[1], sharex=ax12)
arrivals.plot(color='gray', ax=ax2, marker=".", linestyle="None", legend=False)

ax2.scatter(x, y, s=z, color='gray')

ticks, labels = list(zip(*y_axis))
ax2.set_yticks(ticks)
ax2.set_yticklabels([l.capitalize().replace("_", " ") for l in labels])
ax2.set_ylim([-0.5, 2])

ax2.set_xlabel("")
plt.show()

In [ ]:
plt.figure(figsize=(15, 5))

container_pickups = df_truck.groupby(
    pd.Grouper(key='realized_container_pickup_time', freq='H')
).count().fillna(0)

scaling_factor = 7
y_pos_scaled = [y_i * scaling_factor for y_i in y]
ax = container_pickups["delivers_container"].plot()
ax.set_title("Number of trucks arriving in each hour that pick up a container")

ax2 = ax.twinx()
ax.scatter(x, y, color='y', s=70)
ticks, labels = list(zip(*y_axis))
ax2.set_yticks(ticks)
ax2.set_yticklabels(labels)

plt.show()

In [ ]:
container_pickups.groupby(container_pickups.index.hour).mean()["picks_up_container"].plot()
plt.title("Container pickups at each hour of the day")
plt.show()

container_deliveries_by_truck.groupby(container_deliveries_by_truck.index.hour).mean()["delivers_container"].plot()
plt.title("Container deliveries at each hour of the day")
plt.show()

This is the probability of the truck to show up at any given hour of the week (by index).

In [ ]:
ax = container_pickups.groupby(container_pickups.index.weekday).mean()["picks_up_container"].plot.bar()
ax.set_xlabel("")
plt.title("Container pickups at each day of the week")
plt.show()

ax = container_deliveries_by_truck.groupby(container_deliveries_by_truck.index.weekday).mean()["delivers_container"].plot.bar()
ax.set_xlabel("")
plt.title("Container deliveries at each day of the week")
plt.show()

In [ ]:
plt.plot([0.0, 0.0, 0.0, 0.0, 0.0, 0.004136582430806258, 0.008498796630565584, 0.007295427196149218,
          0.008348375451263539, 0.011131167268351384, 0.01286101083032491, 0.015418170878459687, 0.012936221419975932,
          0.015568592057761732, 0.01391395908543923, 0.011732851985559567, 0.013161853188929002, 0.011206377858002407,
          0.008498796630565584, 0.007069795427196149, 0.0053399518652226235, 0.003309265944645006, 0.00210589651022864,
          0.002331528279181709, 0.0019554753309265946, 0.002030685920577617, 0.0017298435619735259, 0.002331528279181709,
          0.0025571600481347776, 0.00631768953068592, 0.009401323706377859, 0.008197954271961492, 0.009852587244283995,
          0.012033694344163659, 0.01338748495788207, 0.01233453670276775, 0.014290012033694344, 0.014666064981949459,
          0.014741275571600482, 0.012409747292418772, 0.01105595667870036, 0.01000300842358604, 0.007295427196149218,
          0.005941636582430806, 0.0053399518652226235, 0.0030836341756919376, 0.0019554753309265946,
          0.0024819494584837547, 0.0018050541516245488, 0.0019554753309265946, 0.0013537906137184115,
          0.0009777376654632973, 0.002331528279181709, 0.007821901323706379, 0.009100481347773767, 0.00947653429602888,
          0.009927797833935019, 0.01263537906137184, 0.011958483754512635, 0.015192539109506619, 0.014741275571600482,
          0.01647111913357401, 0.015342960288808664, 0.01233453670276775, 0.011883273164861612, 0.009626955475330927,
          0.008348375451263539, 0.006167268351383875, 0.004362214199759326, 0.004061371841155234, 0.002331528279181709,
          0.002331528279181709, 0.001128158844765343, 0.0012785800240673888, 0.00105294825511432, 0.0009025270758122744,
          0.0018802647412755715, 0.005490373044524669, 0.01022864019253911, 0.011206377858002407, 0.01105595667870036,
          0.01210890493381468, 0.015568592057761732, 0.01654632972322503, 0.019780385078219012, 0.018050541516245487,
          0.01782490974729242, 0.016095066185318894, 0.014064380264741275, 0.012560168471720819, 0.008047533092659447,
          0.006468110709987966, 0.006092057761732852, 0.0042870036101083035, 0.003234055354993983, 0.00157942238267148,
          0.002707581227436823, 0.002030685920577617, 0.002331528279181709, 0.00157942238267148, 0.0030084235860409147,
          0.00631768953068592, 0.009551744885679904, 0.01210890493381468, 0.011356799037304452, 0.016170276774969915,
          0.01647111913357401, 0.01752406738868833, 0.01759927797833935, 0.01752406738868833, 0.015944645006016847,
          0.014365222623345367, 0.01210890493381468, 0.008874849578820699, 0.009551744885679904, 0.004512635379061372,
          0.004663056558363418, 0.0029332129963898917, 0.0017298435619735259, 0.002331528279181709,
          0.0025571600481347776, 0.00210589651022864, 0.0019554753309265946, 0.0009777376654632973,
          0.0009777376654632973, 0.001128158844765343, 0.0013537906137184115, 0.001654632972322503, 0.001654632972322503,
          0.0013537906137184115, 0.0006768953068592057, 0.00052647412755716, 0.0004512635379061372, 0.0, 0.0, 0.0, 0.0,
          0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] + ([0] * 24)
)
plt.xlim([0, 168])
plt.show()

In [ ]:
df_truck.loc[
    df_truck["realized_container_pickup_time"].dt.weekday == 6
]

In [ ]:
df_truck.loc[
    df_truck["realized_container_delivery_time"].dt.weekday == 6
]

How much percent is that?

In [ ]:
len(df_truck.loc[
    df_truck["realized_container_delivery_time"].dt.weekday == 6
]) / len(df_truck.loc[
    df_truck["realized_container_delivery_time"].dt.weekday != 6
]) * 100

In [ ]:
delivered_and_picked_up_by_large_vessels_df = df_containers.loc[
    ~pd.isna(df_containers["picked_up_by_large_scheduled_vehicle"])
].join(
    df_large_scheduled_vehicle, on="picked_up_by_large_scheduled_vehicle", rsuffix="_picked_up"
).loc[
    ~pd.isna(df_containers["delivered_by_large_scheduled_vehicle"])
].join(
    df_large_scheduled_vehicle, on="delivered_by_large_scheduled_vehicle", rsuffix="_delivered_by"
)

delivered_and_picked_up_by_large_vessels_df

In [ ]:
dwell_time = (
    delivered_and_picked_up_by_large_vessels_df["scheduled_arrival"]
    - delivered_and_picked_up_by_large_vessels_df["scheduled_arrival_delivered_by"]
)
dwell_time

In [ ]:
dwell_time.describe()

In [ ]:
dwell_time.astype("timedelta64[h]").plot.hist(bins=30, color="gray")
plt.xlabel("Hours between delivery and onward transportation (except trucks)")
plt.ylabel("Number container in July")
plt.show()